Read in data:

In [ ]:
import pandas as pd
import string
import re

df = pd.read_csv('../webscraper/raw_data.csv')

df.head()  


,URL,HTML
0,https://deepmind.google/research/publications/,Publications - Google DeepMind ...
1,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...
2,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...
3,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...
4,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...


Clean the abstacts:

In [ ]:

abstracts = df['HTML'].tolist()
abstracts = [' '.join(abstract.lower().translate(str.maketrans('', '', string.punctuation)).split()) for abstract in abstracts]

Scaling Pre-training to One Hundred Billion Data for Vision Language Models - Google DeepMind                                      Jump to Content                     Google     DeepMind       Search...                 Search Close                            Google     DeepMind                                      About                                                      Learn about Google DeepMind                                             — Our mission is to build AI responsibly to benefit humanity                             Responsibility & Safety                                             — We want AI to benefit the world, so we must be thoughtful about how it’s built and used                             Education                                             — Our vision is to help make the AI ecosystem more representative of society                             Careers                                             — Many disciplines, one common goal                               R

In [ ]:

# regex to extract from HTML
def extract_abstract(text):
    match = re.search(r'(?i)abstract\s*(.*?)(?=\s*authors)', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None
def extract_publish_date(text):
    match = re.search(r'(?i)published\s+([0-9]{1,2}\s+\w+\s+[0-9]{4})', text)
    if match:
        return match.group(1).strip()
    return None
def extract_authors(text):
    match = re.search(r'Authors\s+(.*?)(?=\s*(External|Venue|Published|Footer|$))', text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    return None
def extract_title(text):
    match = re.search(r'([A-Z][^\n]{10,100}?)\s+Published', text)
    if match:
        return match.group(1).strip()
    return None


In [149]:
df['title'] = df['HTML'].apply(extract_title)
df['cleaned_abstract'] = df['HTML'].apply(extract_abstract)
df['publish_date'] = df['HTML'].apply(extract_publish_date)
df['authors'] = df['HTML'].apply(extract_authors)


In [150]:
# checking extractions
print(df['URL'][3])
print(df['title'][3])
print(df['authors'][3])
print(df['cleaned_abstract'][3])
print(df['publish_date'][3])

https://deepmind.google/research/publications/132991
Scaling Pre-training to One Hundred Billion Data for Vision Language Models
Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz, Zhe Li, Keran Rong, Xiaohua Zhai*  *
We provide an empirical investigation of the potential of pre-training vision-language models on an unprecedented scale: 100 billion examples. We find that model performance tends to saturate at this scale on many common Western-centric classification and retrieval benchmarks, such as COCO Captions. Nevertheless, tasks of cultural diversity achieve more substantial gains from the 100-billion scale web data, thanks to its coverage of long-tail concepts. Furthermore, we analyze the model's multilinguality and show gains in low-resource languages as well. In addition, we observe that reducing the size of the pretraining dataset via quality filters like using CLIP, typically used to enhance performance, may inadvertently reduce the cultural diversity represented even in large-scal

In [156]:
df.head(5)

,URL,HTML,cleaned_abstract,publish_date,authors,title
0,https://deepmind.google/research/publications/,Publications - Google DeepMind ...,None,None,Marcus Hutter,None
1,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...,None,None,"A. Feder Cooper, Christopher A. Choquette-Choo...",None
2,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...,A principal goal of computational neuroscience...,6 February 2025,"Pablo Castro Rivadeneira, Kim Stachenfeld, Kev...",Automated Discovery of Interpretable Cognitive...
3,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...,We provide an empirical investigation of the p...,11 February 2025,"Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz,...",Scaling Pre-training to One Hundred Billion Da...
4,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...,None,None,Marcus Hutter,None


In [ ]:
df = df.dropna(subset=['cleaned_abstract'])

df.shape

(278, 6)

In [ ]:
df.to_csv('cleaned_url_data.csv', index=False)
